## Spotipy installation and testing guidelines:

Quick setup:

https://medium.com/@maxtingle/getting-started-with-spotifys-api-spotipy-197c3dc6353b

Overall spotipy documentation:

https://spotipy.readthedocs.io/en/2.13.0/#

1. Install spotipy:

In [20]:
!pip install spotipy

2. Import client ID:

Steps to be followed to create project here: https://developer.spotify.com/dashboard/login

Create spotify account and sign up for developer account and create an application

In [21]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = '49789426e6c04428a2befbd3c2d79b02'
secret = 'd18032252a0d4634aa8ca21356894e79'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

3. Retreive sample data:

Spotify methods to access Artist, Track, and Album data endpoints all require you to know their individual Spotify ID. I started with the search endpoint, exemplified below, because it does not require a Spotify ID.
The following code collects 1,000 Track IDs and their associated track name, artist name, and popularity score.


In [22]:
artist_name = []
track_name = []
popularity = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2018', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

4. Load data in for analysis

Requires pandas installation

In [5]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 9.1 MB/s eta 0:00:0000:0100:01
  Using cached pytz-2022.7.1-py2.py3-none-any.whl (499 kB)


In [23]:
import pandas as pd

track_dataframe = pd.DataFrame({'artist_name' : artist_name, 'track_name' : track_name, 'track_id' : track_id, 'popularity' : popularity})
print(track_dataframe.shape)
track_dataframe.head()

(1000, 4)


,artist_name,track_name,track_id,popularity
0,Morgan Wallen,Whiskey Glasses,6foY66mWZN0pSRjZ408c00,79
1,Morgan Wallen,Chasin' You,5MwynWK9s4hlyKHqhkNn4A,79
2,Juice WRLD,All Girls Are The Same,4VXIryQMWpIdGgYR4TrjT1,84
3,Lil Baby,Drip Too Hard (Lil Baby & Gunna),78QR3Wp35dqAhFEc2qAGjE,82
4,The Weeknd,I Was Never There,1cKHdTo9u0ZymJdPGSh6nq,87


Additional requirements could include getting long term authorization code in the case that this is a deployed application:

- https://developer.spotify.com/documentation/general/guides/authorization/
- https://developer.spotify.com/documentation/general/guides/authorization/code-flow/

***Note that this is more for an application pipeline. For the purposes of collecting labels, this may not be necessary***

Getting audio features API: https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features

In [24]:
import requests

# Step 1: Obtain an access token
auth_url = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(
    auth_url,
    data={
        'grant_type': 'client_credentials',
        'client_id': cid,
        'client_secret': secret
    }
)

auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']

print(access_token)

headers = {
    'Authorization': f'Bearer {access_token}'
}

# Step 2: Get the user's ID
user_id = 'by6gio94n7zv678vo8bvip3nf'

# Step 3: Fetch the user's playlists
playlists_url = f'https://api.spotify.com/v1/users/{user_id}/playlists'
playlists_response = requests.get(playlists_url, headers=headers)
playlists_data = playlists_response.json()

print(playlists_data.keys())

# Step 4: Iterate through each playlist and fetch the songs
for playlist in playlists_data['items']:
    playlist_id = playlist['id']
    playlist_name = playlist['name']
    print(f'Playlist: {playlist_name}')
    
    tracks_url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    tracks_response = requests.get(tracks_url, headers=headers)
    tracks_data = tracks_response.json()

    for track in tracks_data['items']:
        track_name = track['track']['name']
        track_artist = track['track']['artists'][0]['name']
        print(f'\t{track_name} - {track_artist}')

BQCEEJJaQSqTZdHgi4MRr_51Cob_A1YmnW5Fd0s4z1DLpgx4DjZ9U6Gu6GXgiOE-LA4hasFrIqAo8p7a5TsXDoWcEBeGcqpC_4C7KLAIYqbR8Vme0vWb
dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])
Playlist: Mastery
	I Got Summer On My Mind - Jay Dunham
	Serve a Boat - Shoreline Mafia
	Fruit Punch - Jay Faded
	Skank N Flex (with Scrufizzer) - Wax Motif
	Bang Bang - Rita Ora
	Midnight Sky - Miley Cyrus
	Memories - Conan Gray
	Jersey - Future
	Ambitionz Az A Ridah - 2Pac
	All Eyez On Me (ft. Big Syke) - 2Pac
	Anywhere For You - Radio Edit - John Martin
	Years - Alesso
	Over - Drake
	My Silver Lining - First Aid Kit
	Money - The Drums
	the BLACK seminole. - Lil Yachty
	X (feat. Future) - 21 Savage
	Kiss Kiss (feat. T-Pain) - Chris Brown
	Twisted! - Aminé
	Jordan Belfort - Wes Walker
	Power Trip (feat. Miguel) - J. Cole
	Immortal - J. Cole
	How We Do - The Game
	Just Can’t Get Enough - Black Eyed Peas
	Soundtrack 2 My Life - Kid Cudi
	Lights Please - J. Cole
	goosebumps - Travis Scott
	Had Eno

#### Label collection for sample user

Create individual dataframes for each of the desired features that we want to predict. Also collect dataframe for all tracks involved. Print total number of collected labels. We will make sure we can request the desired tracks before adding them on to our final csv file. We can do this process for multiple users and this will be our entire data collection process. If we can grab audio, we can grab tempo and such using librosa. 

In [27]:
import requests

# Step 1: Obtain an access token
auth_url = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(
    auth_url,
    data={
        'grant_type': 'client_credentials',
        'client_id': cid,
        'client_secret': secret
    }
)

auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']

print(access_token)

headers = {
    'Authorization': f'Bearer {access_token}'
}

# Step 2: Get the user's ID
user_id = 'by6gio94n7zv678vo8bvip3nf'

# Step 3: Fetch the user's playlists
playlists_url = f'https://api.spotify.com/v1/users/{user_id}/playlists'
playlists_response = requests.get(playlists_url, headers=headers)
playlists_data = playlists_response.json()

print(playlists_data.keys())

# Set a threshold for desired features
danceability_threshold = 0.85
non_danceability_threshold = 0.25

instrumentalness_threshold = 0.85
non_instrumentalness_threshold = 0.25

speechiness_threshold = 0.85
non_speechiness_threshold = 0.25

acousticness_threshold = 0.85
non_acousticness_threshold = 0.25

energy_threshold = 0.85
non_energy_threshold = 0.25


# Step 4: Iterate through each playlist and fetch the songs
total_danceable = 0
total_non_danceable = 0

total_instrumental = 0
total_non_instrumental = 0

total_speechy = 0
total_non_speechy = 0

total_acoustic = 0
total_non_acoustic = 0

total_energetic = 0
total_non_energetic = 0

total = 0

# Create an empty DataFrame with the appropriate columns
danceable_tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'danceability'])
non_danceable_tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'danceability'])

instrumental_tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'instrumentalness'])
non_instrumental_tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'instrumentalness'])

speechy_tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'speechiness'])
non_speechy_tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'speechiness'])

acoustic_tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'acousticness'])
non_acoustic_tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'acousticness'])

energetic_tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'energy'])
non_energetic_tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'energy'])

all_tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'danceability', 'instrumentalness', 'speechiness', 'acousticness', 'energy', 'valence', 'tempo', 'loudness', 'liveness', 'key', 'mode', 'time_signature'])

for playlist in playlists_data['items']:
    playlist_id = playlist['id']
    playlist_name = playlist['name']
    print(f'Playlist: {playlist_name}')
    
    tracks_url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    tracks_response = requests.get(tracks_url, headers=headers)
    tracks_data = tracks_response.json()

    for track in tracks_data['items']:
        track_id = track['track']['id']
        track_name = track['track']['name']
        track_artist = track['track']['artists'][0]['name']
        
        
        if track_id not in all_tracks_df['track_id'].values:
            # Fetch track's audio features
            audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
            audio_features_response = requests.get(audio_features_url, headers=headers)
            audio_features_data = audio_features_response.json()
            
            # add to all 
            track_df = pd.DataFrame({
                'track_id': [track_id],
                'track_name': [track_name],
                'track_artist': [track_artist],
                'danceability': [audio_features_data['danceability']],
                'instrumentalness': [audio_features_data['instrumentalness']],
                'speechiness': [audio_features_data['speechiness']],
                'acousticness': [audio_features_data['acousticness']],
                'energy': [audio_features_data['energy']],
                'valence': [audio_features_data['valence']],
                'tempo': [audio_features_data['tempo']],
                'loudness': [audio_features_data['loudness']],
                'liveness': [audio_features_data['liveness']],
                'key': [audio_features_data['key']],
                'mode': [audio_features_data['mode']],
                'time_signature': [audio_features_data['time_signature']]
            })
            all_tracks_df = pd.concat([all_tracks_df, track_df], ignore_index=True)
            total = total + 1
        
        
        # Check if the track is already in the DataFrame
        if track_id not in danceable_tracks_df['track_id'].values:
            
            # Fetch track's audio features
            audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
            audio_features_response = requests.get(audio_features_url, headers=headers)
            audio_features_data = audio_features_response.json()

            # Check if the track's danceability is greater than the threshold
            if audio_features_data['danceability'] > danceability_threshold:
                print(f'\t{track_name} - {track_artist} (Danceability: {audio_features_data["danceability"]})')
                # Append the new track to the DataFrame
                track_df = pd.DataFrame({
                    'track_id': [track_id],
                    'track_name': [track_name],
                    'track_artist': [track_artist],
                    'danceability': [audio_features_data['danceability']]
                })
                
                # Concatenate the new DataFrame with the existing DataFrame
                danceable_tracks_df = pd.concat([danceable_tracks_df, track_df], ignore_index=True)
                
                total_danceable = total_danceable + 1
                
        # Check if the track is already in the DataFrame
        if track_id not in non_danceable_tracks_df['track_id'].values:
        
            # Fetch track's audio features
            audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
            audio_features_response = requests.get(audio_features_url, headers=headers)
            audio_features_data = audio_features_response.json()

            # Check if the track's danceability is less than the threshold
            if audio_features_data['danceability'] < non_danceability_threshold:
                print(f'\t{track_name} - {track_artist} (Danceability: {audio_features_data["danceability"]})')
                # Append the new track to the DataFrame
                track_df = pd.DataFrame({
                    'track_id': [track_id],
                    'track_name': [track_name],
                    'track_artist': [track_artist],
                    'danceability': [audio_features_data['danceability']]
                })
                
                # Concatenate the new DataFrame with the existing DataFrame
                non_danceable_tracks_df = pd.concat([danceable_tracks_df, track_df], ignore_index=True)
                
                total_non_danceable = total_non_danceable + 1
                
        # Check if the track is already in the DataFrame
        if track_id not in instrumental_tracks_df['track_id'].values:
        
            # Fetch track's audio features
            audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
            audio_features_response = requests.get(audio_features_url, headers=headers)
            audio_features_data = audio_features_response.json()

            # Check if the track's instrumentalness is greater than the threshold
            if audio_features_data['instrumentalness'] > instrumentalness_threshold:
                print(f'\t{track_name} - {track_artist} (Instrumentalness: {audio_features_data["instrumentalness"]})')
                # Append the new track to the DataFrame
                track_df = pd.DataFrame({
                    'track_id': [track_id],
                    'track_name': [track_name],
                    'track_artist': [track_artist],
                    'instrumentalness': [audio_features_data['instrumentalness']]
                })
                
                # Concatenate the new DataFrame with the existing DataFrame
                instrumental_tracks_df = pd.concat([instrumental_tracks_df, track_df], ignore_index=True)
                
                total_instrumental = total_instrumental + 1
                
        # Check if the track is already in the DataFrame
        if track_id not in non_instrumental_tracks_df['track_id'].values:
        
            # Fetch track's audio features
            audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
            audio_features_response = requests.get(audio_features_url, headers=headers)
            audio_features_data = audio_features_response.json()

            # Check if the track's instrumentalness is less than the threshold
            if audio_features_data['instrumentalness'] < non_instrumentalness_threshold:
                print(f'\t{track_name} - {track_artist} (Instrumentalness: {audio_features_data["instrumentalness"]})')
                # Append the new track to the DataFrame
                track_df = pd.DataFrame({
                    'track_id': [track_id],
                    'track_name': [track_name],
                    'track_artist': [track_artist],
                    'instrumentalness': [audio_features_data['instrumentalness']]
                })
                
                # Concatenate the new DataFrame with the existing DataFrame
                non_instrumental_tracks_df = pd.concat([instrumental_tracks_df, track_df], ignore_index=True)
                
                total_non_instrumental = total_non_instrumental + 1
                
        # Check if the track is already in the DataFrame
        if track_id not in speechy_tracks_df['track_id'].values:
        
            # Fetch track's audio features
            audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
            audio_features_response = requests.get(audio_features_url, headers=headers)
            audio_features_data = audio_features_response.json()

            # Check if the track's speechiness is greater than the threshold
            if audio_features_data['speechiness'] > speechiness_threshold:
                print(f'\t{track_name} - {track_artist} (Speechiness: {audio_features_data["speechiness"]})')
                # Append the new track to the DataFrame
                track_df = pd.DataFrame({
                    'track_id': [track_id],
                    'track_name': [track_name],
                    'track_artist': [track_artist],
                    'speechiness': [audio_features_data['speechiness']]
                })
                
                # Concatenate the new DataFrame with the existing DataFrame
                speecy_tracks_df = pd.concat([speecy_tracks_df, track_df], ignore_index=True)
                
                total_speechy = total_speechy + 1
                
        if track_id not in non_speechy_tracks_df['track_id'].values:
            
            # Fetch track's audio features
            audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
            audio_features_response = requests.get(audio_features_url, headers=headers)
            audio_features_data = audio_features_response.json()
            
            # Check if the track's speechiness is less than the threshold
            if audio_features_data['speechiness'] < non_speechiness_threshold:
                print(f'\t{track_name} - {track_artist} (Speechiness: {audio_features_data["speechiness"]})')
                # Append the new track to the DataFrame
                track_df = pd.DataFrame({
                    'track_id': [track_id],
                    'track_name': [track_name],
                    'track_artist': [track_artist],
                    'speechiness': [audio_features_data['speechiness']]
                })
                
                # Concatenate the new DataFrame with the existing DataFrame
                non_speecy_tracks_df = pd.concat([non_speecy_tracks_df, track_df], ignore_index=True)
                
                total_non_speechy = total_non_speechy + 1
                
        if track_id not in acoustic_tracks_df['track_id'].values:
            
            # Fetch track's audio features
            audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
            audio_features_response = requests.get(audio_features_url, headers=headers)
            audio_features_data = audio_features_response.json()
            
            # Check if the track's acousticness is greater than the threshold
            if audio_features_data['acousticness'] > acousticness_threshold:
                print(f'\t{track_name} - {track_artist} (Acousticness: {audio_features_data["acousticness"]})')
                # Append the new track to the DataFrame
                track_df = pd.DataFrame({
                    'track_id': [track_id],
                    'track_name': [track_name],
                    'track_artist': [track_artist],
                    'acousticness': [audio_features_data['acousticness']]
                })
                
                # Concatenate the new DataFrame with the existing DataFrame
                acoustic_tracks_df = pd.concat([acoustic_tracks_df, track_df], ignore_index=True)
                
                total_acoustic = total_acoustic + 1
                
        if track_id not in non_acoustic_tracks_df['track_id'].values:
            
            # Fetch track's audio features
            audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
            audio_features_response = requests.get(audio_features_url, headers=headers)
            audio_features_data = audio_features_response.json()
            
            # Check if the track's acousticness is less than the threshold
            if audio_features_data['acousticness'] < non_acousticness_threshold:
                print(f'\t{track_name} - {track_artist} (Acousticness: {audio_features_data["acousticness"]})')
                # Append the new track to the DataFrame
                track_df = pd.DataFrame({
                    'track_id': [track_id],
                    'track_name': [track_name],
                    'track_artist': [track_artist],
                    'acousticness': [audio_features_data['acousticness']]
                })
                
                # Concatenate the new DataFrame with the existing DataFrame
                non_acoustic_tracks_df = pd.concat([non_acoustic_tracks_df, track_df], ignore_index=True)
                
                total_non_acoustic = total_non_acoustic + 1
                
        if track_id not in energetic_tracks_df['track_id'].values:
            
            # Fetch track's audio features
            audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
            audio_features_response = requests.get(audio_features_url, headers=headers)
            audio_features_data = audio_features_response.json()
            
            # Check if the track's energy is greater than the threshold
            if audio_features_data['energy'] > energy_threshold:
                print(f'\t{track_name} - {track_artist} (Energy: {audio_features_data["energy"]})')
                # Append the new track to the DataFrame
                track_df = pd.DataFrame({
                    'track_id': [track_id],
                    'track_name': [track_name],
                    'track_artist': [track_artist],
                    'energy': [audio_features_data['energy']]
                })
                
                # Concatenate the new DataFrame with the existing DataFrame
                energetic_tracks_df = pd.concat([energetic_tracks_df, track_df], ignore_index=True)
                
                total_energetic = total_energetic + 1
                
                
        if track_id not in non_energetic_tracks_df['track_id'].values:
            
            # Fetch track's audio features
            audio_features_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
            audio_features_response = requests.get(audio_features_url, headers=headers)
            audio_features_data = audio_features_response.json()
            
            # Check if the track's energy is less than the threshold
            if audio_features_data['energy'] < non_energy_threshold:
                print(f'\t{track_name} - {track_artist} (Energy: {audio_features_data["energy"]})')
                # Append the new track to the DataFrame
                track_df = pd.DataFrame({
                    'track_id': [track_id],
                    'track_name': [track_name],
                    'track_artist': [track_artist],
                    'energy': [audio_features_data['energy']]
                })
                
                # Concatenate the new DataFrame with the existing DataFrame
                non_energetic_tracks_df = pd.concat([non_energetic_tracks_df, track_df], ignore_index=True)
                
                total_non_energetic = total_non_energetic + 1

           
print(f'Total tracks: {total}')
print(f'Total danceable tracks: {total_danceable}')
print(f'Total non-danceable tracks: {total_non_danceable}')
print(f'Total speecy tracks: {total_speechy}')
print(f'Total non-speecy tracks: {total_non_speechy}')
print(f'Total acoustic tracks: {total_acoustic}')
print(f'Total non-acoustic tracks: {total_non_acoustic}')
print(f'Total energetic tracks: {total_energetic}')
print(f'Total non-energetic tracks: {total_non_energetic}')
print(f'Total instrumental tracks: {total_instrumental}')
print(f'Total non-instrumental tracks: {total_non_instrumental}')

# # Display the DataFrame with danceable tracks
# print(danceable_tracks_df)

BQBAGVhgp7hGIgf9RvtfOiVi0jHhg-zqd96_5E3AscPQEFa5Q4TdFv4ZftgL1gaWvR66u-3alMPcwAfOZxTBMk3ST5J4yP2b5sGWTHZ3rE4JTRmITfF5
dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])
Playlist: Mastery
	I Got Summer On My Mind - Jay Dunham (Speechiness: 0.0631)
	I Got Summer On My Mind - Jay Dunham (Acousticness: 0.00148)
	Serve a Boat - Shoreline Mafia (Instrumentalness: 0)
	Serve a Boat - Shoreline Mafia (Speechiness: 0.226)
	Serve a Boat - Shoreline Mafia (Acousticness: 0.0312)
	Fruit Punch - Jay Faded (Instrumentalness: 0.0218)
	Fruit Punch - Jay Faded (Speechiness: 0.0806)
	Fruit Punch - Jay Faded (Acousticness: 0.182)
	Skank N Flex (with Scrufizzer) - Wax Motif (Danceability: 0.969)
	Skank N Flex (with Scrufizzer) - Wax Motif (Instrumentalness: 0.00593)
	Skank N Flex (with Scrufizzer) - Wax Motif (Speechiness: 0.123)
	Skank N Flex (with Scrufizzer) - Wax Motif (Acousticness: 0.00416)
	Skank N Flex (with Scrufizzer) - Wax Motif (Energy: 0.87)
	Bang Bang - Rita Ora (Instr

In [ ]:
non_danceable_tracks_df = pd.DataFrame(columns=['track_id', 'track_name', 'track_artist', 'danceability'])

